In [13]:
import pandas as pd
import pymorphy2
import re
from collections import defaultdict
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
cat = ['политика', 'россия', 'сша', 'европа', 'экономика', 'общество', 'преступность и право', 'происшествия', 'культура', 'интернет']

df = pd.read_pickle('data/dftime_cat.pkl')


print(df.info()) 
print(df.shape)
print(df.sample(3))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1906 entries, 0 to 11727
Data columns (total 4 columns):
id        1906 non-null int64
title     1906 non-null object
cat_id    1906 non-null int64
text2     1906 non-null object
dtypes: int64(2), object(2)
memory usage: 74.5+ KB
None
(1906, 4)
         id                                              title  cat_id  \
721    4473  экс-премьер пакистана беназир бхутто взята под...       0   
184    2201         пилотируемые полёты должны иметь приоритет       2   
2716  16077  президент рф наградил выдающихся деятелей куль...       1   

                                                  text2  
721    пакистан, премьер-министр пакистана в 1988—19...  
184   космонавтикапилотируемые полёты должны иметь п...  
2716  культурав преддверии дня народного единства пр...  


In [6]:
morph = pymorphy2.MorphAnalyzer()

def normal_word(w):
    ''' '''
    w = w.lower()
    info = morph.parse(w)
    if(len(info) > 0):
        return info[0].normal_form
    return w

def word_extraction(sentence):  
    ''' '''
    ignore = {'и','в','а','с','о','к','у','ли', 'можно', 'на', 'снова', 'вот','что','как','без','по','считать','свой',
             'который','два','она','это','она','для','тот','если', 'то', 'такой','от', 'он', 'за', 'из','до','быть',
             'об', 'этом' , 'так', 'его', 'после', 'вновь', 'все', 'а','с', 'ч', 'п', 'в', 'n', 'я', 'a', 'у', 'м', 'й', 
              'т', 'h', 'x', 'е', 'и', 'r', 'н', 'g', 'о', 'm', 'c', 'а', 'к', 't', 'l','стало', 'стал'
             }    
    sentence = re.sub("(не)\s+", "\g<1>", sentence)
    sentence = sentence.replace("ё", "е")    
    sentence = re.sub("[\d\.\-«,%»\"\(\)—]", " ", sentence)
    words = re.sub("[^\w]", " ",  sentence).split() 

    cleaned_text = []
    for w in words:
        w =  normal_word(w)
        if w not in ignore:
            cleaned_text.append(w)        
    return cleaned_text

def text_to_token(texts):
    ''' '''
    text_words = []
    
    frequency = defaultdict(int)
    for text in texts:
        words = word_extraction(text)
        text_words.append(words)
        for token in words:
            frequency[token] += 1

    return [
        [token for token in words if frequency[token] > 1]
            for words in text_words
    ]


def processed_text(df):
    texts = df['text2']
    return text_to_token(texts)


texts = processed_text(df)

In [14]:
texts_str = [" ".join(text) for text in texts]
vectorizer = TfidfVectorizer(ngram_range = (1,2), min_df = 3)  
X = vectorizer.fit_transform(texts_str)

In [15]:
def cat_id_vec(id):
    v = np.zeros(10)
    v[id] = 1
    return v
Y = df['cat_id'].map(cat_id_vec).values

Y_ = df['cat_id'].values
Y_label = df['cat_id'].values

In [18]:
from sklearn.decomposition import TruncatedSVD

def encode_svd(x, k=100): 
    svd_model = TruncatedSVD(n_components=k, algorithm='randomized', n_iter=100, random_state=42)
    x1 = svd_model.fit_transform(X) 
    return x1 

In [19]:
%%time
X_ = encode_svd(X)

CPU times: user 53.5 s, sys: 57.7 s, total: 1min 51s
Wall time: 33.4 s


In [20]:
np.save('data_x.npy', X_)
np.save('data_y.npy', Y_)